# Бот торгует по стратегии входа в сделку по индикатору RSI. Данные не потоковые.

In [1]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.um_futures import UMFutures
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keys
import time
import logging
import requests
import talib 
from talib import abstract
from tqdm import tqdm, tqdm_notebook
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
pd.options.mode.chained_assignment = None # отключение дебильного предупреждения

# Подключение к бирже

In [2]:
client = Client(keys.api_key, keys.secret_key)

# Задать переменные

In [3]:
#coins = ['BELUSDT', 'BNBUSDT', '1INCHUSDT', 'C98USDT', 'CHRUSDT', 'CHZUSDT', 'CTKUSDT', 'DOGEUSDT', 'EGLDUSDT', 'ENSUSDT','ETHUSDT', 'KSMUSDT', 
#             'LINKUSDT', 'LTCUSDT', 'MASKUSDT', 'MATICUSDT', 'OCEANUSDT', 'OPUSDT', 'PEOPLEUSDT', 'RENUSDT', 'RLCUSDT']

In [4]:
ПОРОГ_RSI_1Ч_ШОРТ = 68 # значение RSI для открытия шорт 1Ч TM
ПОРОГ_RSI_5M_ШОРТ = 68 # значение RSI для открытия шорт 5М TM
ПОРОГ_RSI_ЗАКРЫТ_ШОРТА = 32  # значение RSI для закрытия шорт
ПОРОГ_RSI_1Ч_ЛОНГ = 32 # значение RSI для открытия лонг 1Ч TM
ПОРОГ_RSI_5M_ЛОНГ = 32  # значение RSI для открытия лонг 5М TM
ПОРОГ_RSI_ЗАКРЫТ_ЛОНГА = 68   # значение RSI для закрытия лонг
WINDOW_SLOW_MA = 100
INTERVAL = Client.KLINE_INTERVAL_5MINUTE  # младший таймфрейм 
INTERVAL_OLD_TM = Client.KLINE_INTERVAL_1HOUR    # старший таймфрейм 
LOOKBACK = '80' # количество минут истории свечей, вводим в ковычках, должно быть больше чем 14 свечей. LOOKBACK = INTERVAL * 14 + 50 СВЕЧЕЙ
PAUSE_BETWEEN_GET_DATA = 10 # период между обновлением данных в секундах
TAKE_PROFIT_PROC = 2.0 #  это проценты
STOP_LOSS_PROC = 0.8   #  это проценты

# Отправить сообщение в tlg

In [5]:
def send_tg_message(text):
    requests.post(
    'https://api.telegram.org/' +
    'bot{}/sendMessage'.format(keys.TOKEN_tlg_bot), 
    params=dict(chat_id=keys.chat_id_tlg, text=text)
    )

# Получение и обработка данных

## Данные старшего таймфрейма

In [6]:
def get_data_older_tm(symbol, interval, lookback):
    klines = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + 'min ago UTC', limit=1000))
    klines.columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'Kline_close_time', 'Quote_asset_volume',
               'Number_of_trades', 'Taker_buy_base_asset_volume', 'Taker_buy_quote_asset_volume', 'ignore']
    klines = klines.iloc[:, :6]
    klines['open_time'] = pd.to_datetime(klines['open_time'], unit='ms')
    float_columns = ['open', 'high', 'low', 'close', 'volume']
    klines[float_columns] = klines[float_columns].astype('float')
    klines['slow_sma'] = klines['close'].rolling(window = WINDOW_SLOW_MA).mean()
    klines['global_trend'] =  np.where((klines['slow_sma'] / klines['slow_sma'].rolling(window = 3).mean()) >= 1, 1, 0)
    return klines

In [7]:
# df_old_tm =  get_data_older_tm('BTCUSDT', interval = '1h', lookback = '6120')

## Данные для трейдов: поиск точки входа, выхода

In [8]:
def get_data_search_point(symbol, interval, lookback):
    klines = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + 'min ago UTC', limit=1000))
    klines.columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'Kline_close_time', 'Quote_asset_volume',
               'Number_of_trades', 'Taker_buy_base_asset_volume', 'Taker_buy_quote_asset_volume', 'ignore']
    klines = klines.iloc[:, :6]
    klines['open_time'] = pd.to_datetime(klines['open_time'], unit='ms')
    float_columns = ['open', 'high', 'low', 'close', 'volume']
    klines[float_columns] = klines[float_columns].astype('float')
    klines['fast_sma'] = klines['close'].rolling(window = 2).mean()
    klines['RSI'] = talib.RSI(klines['close'], timeperiod = 14)
    #klines['trend_power'] =  klines['fast_sma']/klines['fast_sma'].shift()
    return klines

In [9]:
# get_data_search_point('BTCUSDT', interval = INTERVAL, lookback = LOOKBACK)

# Построить график

Построить график цены:

In [10]:
def build_graph(data):    
    plt.figure(figsize=(13, 3))
    plt.plot(data['open_time'], data['close'],  label = 'Close_price')
    plt.ylabel('Close_price USDT')
    plt.xlim(data['open_time'].iloc[0], data['open_time'].iloc[-1])
    plt.legend(loc = 'upper left')
    plt.grid()
    return plt.show() 

In [11]:
#build_graph(get_data_search_point('BTCUSDT', interval = INTERVAL, lookback = LOOKBACK))

Построить график RSI

In [12]:
def build_graph_RSI(data):    
    plt.figure(figsize=(13, 3))
    plt.plot(data['open_time'], data['RSI'], label = 'RSI_14')
    plt.ylabel('RSI_14')
    plt.ylim(0, 100)
    plt.xlim(data['open_time'].iloc[0], data['open_time'].iloc[-1])
    plt.legend(loc = 'upper left')
    plt.grid()
    return plt.show() 

In [13]:
def build_graph_TR_POW(data):    
    plt.figure(figsize=(13, 3))
    plt.plot(data['open_time'], data['trend_power'], label = 'Trend_power', color='g')
    plt.ylabel('Trend_power')
    plt.ylim(0.99700, 1.00300)
    plt.xlim(data['open_time'].iloc[0], data['open_time'].iloc[-1])
    plt.legend(loc = 'upper left')
    plt.grid()
    return plt.show() 

In [14]:
#build_graph_TR_POW(get_data_search_point('BTCUSDT', interval = INTERVAL, lookback = LOOKBACK))

# Получить текущее время

In [15]:
def get_time():
    t = time.localtime() 
    current_time = time.strftime("%Y-%m-%d__%H:%M:%S", t)
    current_time = pd.to_datetime(current_time, format='%Y-%m-%d__%H:%M:%S')
    return current_time

# Получить текущую цену монеты

In [16]:
def get_price(coin):  # coin - тикер монеты
    df = pd.DataFrame(client.get_all_tickers())
    price_coin = df[df['symbol'] == coin]['price'].values[0]
    price_coin = pd.to_numeric(price_coin)
    return price_coin

# Опреции со статистикой

In [17]:
coin_st = []
side_position_st = []
date_entry_st = []
date_exit_st = []
profit_st = []
rsi_5m_st = []
rsi_1h_st = []

## Собрать статистику

In [18]:
def collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H):
    coin_st.append(coin)
    side_position_st.append(side_position)
    date_entry_st.append(date_entry)
    date_exit_st.append(date_exit)
    profit_st.append(profit)
    rsi_5m_st.append(RSI_5M)
    rsi_1h_st.append(RSI_1H)
    statistics = pd.DataFrame({'Монета': coin_st,
                               'Направление': side_position_st,
                               'Дата входа': date_entry_st,
                               'Дата выхода': date_exit_st,
                               'Прибыль': profit_st,
                               'RSI_5M при входе': rsi_5m_st,
                               'RSI_1H при входе': rsi_1h_st
                                })
    statistics.to_csv('C:/Users/1/Desktop/Proj/statistic.csv', index_label=False)
    #statistics.to_csv('C:/Users/regio/Desktop/Робот/statisitic.csv', index_label=False)
    return statistics

## Обнулить статистику

In [20]:
def clear_statistics():
    stat = pd.read_csv('C:/Users/1/Desktop/Proj/statistic.csv')
    #stat = pd.read_csv('C:/Users/regio/Desktop/Робот/statisitic.csv')
    stat.drop(stat.index, inplace =True)
    stat.to_csv('C:/Users/1/Desktop/Proj/statistic.csv', index_label=False)
    #stat.to_csv('C:/Users/regio/Desktop/Робот/statisitic.csv', index_label=False)
    return stat    

Чтобы обнулить статистику - убрать решетку в начале строки и запустить ячейку. Решетку нужно вернуть обратно!!!

In [21]:
#clear_statistics() 

# Парсинг RSI

In [22]:
url  = 'https://ru.tradingview.com/chart/RK3tIB3E/'

Запуск браузера

In [23]:
options_chrome = webdriver.ChromeOptions()
options_chrome.add_argument('user-data-dir=C:\\Users\\1\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
options_chrome.add_argument("--headless")
browser = webdriver.Chrome(options = options_chrome)
browser.get(url)
# headless - скрыть браузер во время работы, указывается в опциях

Функция для изменения текущей монеты в трейдингвью

In [24]:
def input_coin(coin):   
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.ID, 'header-toolbar-symbol-search'))).click()
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="overlap-manager-root"]/div/div/div[2]/div/div[2]/div[1]/input'))).send_keys(coin, Keys.RETURN)
    time.sleep(0.5)
    return coin

In [25]:
#input_coin('ADAUSDT')

Смена таймфрейма на 1Ч

In [26]:
def change_TM_1H():
    a = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="header-toolbar-intervals"]/div/button[2]'))).click()
    return a

In [27]:
#change_TM_1H()

Смена таймфрейма на 5М

In [28]:
def change_TM_5M():
    a = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="header-toolbar-intervals"]/div/button[1]'))).click()
    return a

In [29]:
#change_TM_5M()

Функция парсинга RSI

In [30]:
def calc_RSI():
    RSI = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[2]/div[1]/div/table/tr[3]/td[2]/div/div[2]/div/div[2]/div[2]/div[2]/div/div[1]/div')))
    RSI = RSI.text
    RSI = float(RSI)
    return RSI

In [31]:
#browser.quit()

In [32]:
#browser.close()

# Расчет STOP-LOSS И TAKE_PROFIT

 условие закрытия позиции, стоп-лосс - умножаем, тэйк - делим

In [33]:
def STOP_LOSS_CALC(STOP_LOSS):
    stop_loss = price_entry * (1 + (STOP_LOSS/100))
    return stop_loss

In [34]:
def TAKE_PROFIT_CALC(TAKE_PROFIT):
    take_profit = price_entry / (1 + TAKE_PROFIT/100)
    return take_profit

In [35]:
def STOP_LOSS_CALC_LONG(STOP_LOSS):
    stop_loss = price_entry / (1 + (STOP_LOSS/100))
    return stop_loss                          

In [36]:
def TAKE_PROFIT_CALC_LONG(TAKE_PROFIT):
    take_profit = price_entry * (1 + TAKE_PROFIT/100)
    return take_profit

# БОТ

Список монет, которыми торгуем

In [37]:
coins = ['BATUSDT', 'APEUSDT', 'NEARUSDT', 'NEOUSDT', 'DUSKUSDT', 'ANKRUSDT', 'COCOSUSDT', 'MKRUSDT', 'YFIUSDT', 'ZRXUSDT', 'LINAUSDT', 
             'DYDXUSDT', 'ALGOUSDT', 'BNXUSDT', 'GMTUSDT', 'COMPUSDT', 'XRPUSDT', 'STORJUSDT', 'ADAUSDT', 'BANDUSDT']

In [38]:
def bot(interval = INTERVAL, interval_old_tm = INTERVAL_OLD_TM, lookback = LOOKBACK, open_position=False, search_point = False): # buy_amt - сумма в $, на которую будем заходить. open_position=False - чтобы не открывать много позиций,
                                                                                                                                                   # если позиция будет открыта, то функция не сработает         
    # qty = round(buy_amt/df.close.iloc[-1], 1) # объем, на который будем заходить(количество монет)
    print('Поиск монеты....')
    for coin in tqdm(coins):
        try:
            input_coin(coin)                                       # делаем монету текущей в трейдингвью  
            time.sleep(1)
            change_TM_5M()
            time.sleep(2)
            RSI_5M = calc_RSI()                                   # получаем RSI по ней  
            change_TM_1H()
            time.sleep(2)
            RSI_1H = calc_RSI()
        except:              
            print('Ошибка парсинга')
            send_tg_message('Ошибка парсинга БОТ 1')
            browser.refresh()
            time.sleep(60)
            input_coin(coin)                                       # делаем монету текущей в трейдингвью                                    
            change_TM_5M()
            time.sleep(2)
            RSI_5M = calc_RSI()                                   # получаем RSI по ней  
            change_TM_1H()
            time.sleep(2)
            RSI_1H = calc_RSI()
        
   #######################################################################################################################################################         
        if RSI_5M > ПОРОГ_RSI_5M_ШОРТ:       # если RSI выше порогового значения               РАЗДЕЛ RSI > 70
            search_point = True  # включение поиска точки входа
            
            while search_point == True:   

                try:
                    df = get_data_search_point(coin, interval, lookback) # получаем историю свечей по ней

                except BinanceAPIException as e:  # обработки ошибок загрузки данных
                    print(e)
                    time.sleep(60)
                    df = get_data_search_point(coin, interval, lookback) # получаем историю свечей по ней              

                if df['RSI'].iloc[-1] >= df['RSI'].iloc[-2]:   #  условие поиска        

                    print(f'Поиск точки входа по монете: {coin}')
                    time.sleep(PAUSE_BETWEEN_GET_DATA)

                elif  df['RSI'].iloc[-1] < df['RSI'].iloc[-2]:  # поиск закончен - вход в позицию

                      #############################################################################################################################################
                    open_position = True
                    side_position = 'Шорт'
                    price_entry = df['close'].iloc[-1]
                    date_entry = get_time()
                    stop_loss = price_entry * (1 + (STOP_LOSS_PROC/100))
                    print(f'Открыта позиция в шорт по монете: {coin}, цена: {price_entry}, время: {date_entry}')
                    send_tg_message(f'Открыта позиция в шорт по монете: {coin}, цена: {price_entry}, время: {date_entry}')
                    # отправка ордера будет сдесь
                    time.sleep(PAUSE_BETWEEN_GET_DATA)
                    search_point = False    # флаг поиск точки входа - отключить

                    while open_position == True: # включение поиска точки выхода

                        try:
                            df = get_data_search_point(coin, interval, lookback) # получаем данные в том же таймфрейме
                            

                        except BinanceAPIException as e:              # для обработки ошибок используем библиотеку от Бинансе
                            print(e)
                            time.sleep(60)
                            df = get_data_search_point(coin, interval, lookback)             
             
                        if df['close'].iloc[-1] >= stop_loss:  # СРАБОТАЛ STOP_LOSS
                            date_exit = get_time()
                            price_exit = df['close'].iloc[-1]
                            profit = -STOP_LOSS_PROC
                            print('Позиция закрыта по STOP_LOSS')
                            print(f'Закрыта позиция в шорт по STOP_LOSS, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                            send_tg_message(f'Закрыта позицию в шорт по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                            collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                            time.sleep(PAUSE_BETWEEN_GET_DATA)

                            open_position = False    # флаг - позиция закрыта
                            
                            
                            
                        elif df['close'].iloc[-1] <= price_entry / (1 + 0.8/100): # CРАБОТАЛ ПРОМЕЖУТ ТЭЙК
                                date_exit = get_time()
                                price_exit = df['close'].iloc[-1]
                                profit = 0.4

                                print('Позиция закрыта по промежуточному ТЭЙКУ')
                                print(f'Закрыта позиция в шорт по промежуточному ТЭЙКУ, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                send_tg_message(f'Закрыта позицию в шорт по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                                time.sleep(PAUSE_BETWEEN_GET_DATA)

                                open_position = False    # флаг - позиция закрыта
                                open_position_1_2 = True
                                
                                while open_position_1_2 == True:
                                    
                                    try:
                                        df = get_data_search_point(coin, interval, lookback) # получаем данные в том же таймфрейме
                                        RSI = calc_RSI()    # парсим RSI

                                    except BinanceAPIException as e:              # для обработки ошибок используем библиотеку от Бинансе
                                        print(e)
                                        time.sleep(60)
                                        df = get_data_search_point(coin, interval, lookback)                        
                                    except:              
                                        print('Ошибка парсинга')
                                        browser.refresh()
                                        time.sleep(60)
                                        input_coin(coin)                                            # делаем монету текущей в трейдингвью
                                        RSI = calc_RSI()                                            # получаем RSI по ней     

                                    if df['close'].iloc[-1] >= stop_loss:  # СРАБОТАЛ STOP_LOSS
                                        date_exit = get_time()
                                        price_exit = df['close'].iloc[-1]
                                        profit = -STOP_LOSS_PROC/2
                                        print('Позиция закрыта по STOP_LOSS')
                                        print(f'Закрыта позиция в шорт по STOP_LOSS, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                        send_tg_message(f'Закрыта позицию в шорт STOP_LOSS по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                        collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                                        open_position_1_2 = False    # флаг - позиция закрыта     
                                    
                                    elif df['close'].iloc[-1] <= price_entry / (1 + (STOP_LOSS_PROC * 3)/100):  # СРАБОТАЛ ТЭЙК
                                        date_exit = get_time()
                                        price_exit = df['close'].iloc[-1]
                                        profit = ((price_entry / price_exit - 1) * 100) / 2
                                        print('Позиция закрыта по TAKE_PROFIT')
                                        print(f'Закрыта позиция в шорт по TAKE_PROFIT, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                        send_tg_message(f'Закрыта позицию в шорт TAKE_PROFIT по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                        collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                                        open_position_1_2 = False    # флаг - позиция закрыта
             
                                        
                                    elif RSI <= ПОРОГ_RSI_ЗАКРЫТ_ШОРТА: # условие закрытия по RSI шортовой сделки

                                        date_exit = get_time()
                                        price_exit = df['close'].iloc[-1]
                                        profit = ((price_entry / price_exit - 1) * 100) / 2
                                        print('Позиция закрыта по RSI')
                                        print(f'Закрыта позиция в шорт по RSI, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                        send_tg_message(f'Закрыта позицию в шорт RSI по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                        collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор статистики

                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                                        open_position_1_2 = False    # флаг - позиция закрыта

                                    else:  
                                        # print('Позиция отерыта в ШОРТ')
                                        open_position_1_2 = True
                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                        else:
                            open_position = True
                            time.sleep(PAUSE_BETWEEN_GET_DATA)
                            # цена между стоп-лосс и первым тэйком

                else:
                    time.sleep(PAUSE_BETWEEN_GET_DATA)
                    print(f'Позиция не открыта по RSI {coin}, время {get_time()}')          
    
    ############################################################################################################################################################
        elif RSI_5M < ПОРОГ_RSI_5M_ЛОНГ:       # если RSI ниже порогового значения     

            search_point = True  # включение поиска точки входа
            
            while search_point == True:   

                try:
                    df = get_data_search_point(coin, interval, lookback) # получаем историю свечей по ней

                except BinanceAPIException as e:  # обработки ошибок загрузки данных
                    print(e)
                    time.sleep(60)
                    df = get_data_search_point(coin, interval, lookback) # получаем историю свечей по ней              

                if df['RSI'].iloc[-1] <= df['RSI'].iloc[-2]:   #  условие поиска        

                    print(f'Поиск точки входа по монете: {coin}')
                    time.sleep(PAUSE_BETWEEN_GET_DATA)

                elif  df['RSI'].iloc[-1] > df['RSI'].iloc[-2]:  # поиск закончен - вход в позицию

                      #############################################################################################################################################
                    open_position = True
                    side_position = 'Лонг'
                    price_entry = df['close'].iloc[-1]
                    date_entry = get_time()
                    stop_loss = price_entry / (1 + (STOP_LOSS_PROC/100))
                    print(f'Открыта позиция в лонг по монете: {coin}, цена: {price_entry}, время: {date_entry}')
                    send_tg_message(f'Открыта позиция в лонг по монете: {coin}, цена: {price_entry}, время: {date_entry}')
                    # отправка ордера будет сдесь
                    time.sleep(PAUSE_BETWEEN_GET_DATA)
                    search_point = False    # флаг поиск точки входа - отключить

                    while open_position == True: # включение поиска точки выхода

                        try:
                            df = get_data_search_point(coin, interval, lookback) # получаем данные в том же таймфрейме
                            

                        except BinanceAPIException as e:              # для обработки ошибок используем библиотеку от Бинансе
                            print(e)
                            time.sleep(60)
                            df = get_data_search_point(coin, interval, lookback)             
             
                        if df['close'].iloc[-1] < stop_loss:  # СРАБОТАЛ STOP_LOSS
                            date_exit = get_time()
                            price_exit = df['close'].iloc[-1]
                            profit = -STOP_LOSS_PROC
                            print('Позиция закрыта по STOP_LOSS')
                            print(f'Закрыта позиция в Лонг по STOP_LOSS, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                            send_tg_message(f'Закрыта позицию в Лонг STOP_LOSS по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                            collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                            time.sleep(PAUSE_BETWEEN_GET_DATA)

                            open_position = False    # флаг - позиция закрыта
                            
                            
                            
                        elif df['close'].iloc[-1] >= price_entry * (1 + 0.8/100): # CРАБОТАЛ ПРОМЕЖУТ ТЭЙК
                                date_exit = get_time()
                                price_exit = df['close'].iloc[-1]
                                profit = 0.4

                                print('Позиция закрыта по промежуточному ТЭЙКУ')
                                print(f'Закрыта позиция в лонг по промежуточному ТЭЙКУ, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                send_tg_message(f'Закрыта позицию в лонг по промежуточному ТЭЙКУ, монета: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                                time.sleep(PAUSE_BETWEEN_GET_DATA)

                                open_position = False    # флаг - позиция закрыта
                                open_position_1_2 = True
                                
                                while open_position_1_2 == True:
                                    
                                    try:
                                        df = get_data_search_point(coin, interval, lookback) # получаем данные в том же таймфрейме
                                        RSI = calc_RSI()    # парсим RSI

                                    except BinanceAPIException as e:              # для обработки ошибок используем библиотеку от Бинансе
                                        print(e)
                                        time.sleep(60)
                                        df = get_data_search_point(coin, interval, lookback)                        
                                    except:              
                                        print('Ошибка парсинга')
                                        browser.refresh()
                                        time.sleep(60)
                                        input_coin(coin)                                            # делаем монету текущей в трейдингвью
                                        RSI = calc_RSI()                                            # получаем RSI по ней     

                                    if df['close'].iloc[-1] <= stop_loss:  # СРАБОТАЛ STOP_LOSS
                                        date_exit = get_time()
                                        price_exit = df['close'].iloc[-1]
                                        profit = -STOP_LOSS_PROC/2
                                        print('Позиция закрыта по STOP_LOSS')
                                        print(f'Закрыта позиция в Лонг по STOP_LOSS, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                        send_tg_message(f'Закрыта позицию в Лонг STOP_LOSS по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                        collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                                        open_position_1_2 = False    # флаг - позиция закрыта     
                                    
                                    elif df['close'].iloc[-1] >= price_entry * (1 + (STOP_LOSS_PROC * 3)/100):  # СРАБОТАЛ ТЭЙК
                                        date_exit = get_time()
                                        price_exit = df['close'].iloc[-1]
                                        profit = ((price_exit / price_entry - 1) * 100) / 2
                                        print('Позиция закрыта по TAKE_PROFIT')
                                        print(f'Закрыта позиция в лонг по TAKE_PROFIT, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                        send_tg_message(f'Закрыта позицию в лонг TAKE_PROFIT по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                        collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор СТАТИСТИКИ

                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                                        open_position_1_2 = False    # флаг - позиция закрыта
             
                                        
                                    elif RSI >= ПОРОГ_RSI_ЗАКРЫТ_ЛОНГА: # условие закрытия по RSI шортовой сделки

                                        date_exit = get_time()
                                        price_exit = df['close'].iloc[-1]
                                        profit = ((price_exit / price_entry - 1) * 100) / 2
                                        print('Позиция закрыта по RSI')
                                        print(f'Закрыта позиция в лонг по RSI, монета: {coin}, цена: {price_exit}, прибыль: {profit}, время: {date_exit}')
                                        send_tg_message(f'Закрыта позицию в лонг RSI по монете: {coin}, время: {date_exit}, цена: {price_exit}, прибыль: {profit}')

                                        collecting_statistics(coin, side_position, date_entry, date_exit, profit, RSI_5M, RSI_1H) # сбор статистики

                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                                        open_position_1_2 = False    # флаг - позиция закрыта

                                    else:  
                                        # print('Позиция отерыта в ШОРТ')
                                        open_position_1_2 = True
                                        time.sleep(PAUSE_BETWEEN_GET_DATA)

                        else:
                            open_position = True
                            time.sleep(PAUSE_BETWEEN_GET_DATA)
                            # цена между стоп-лосс и первым тэйком

                else:
                    print(f'Позиция не открыта по RSI {coin}, время {get_time()}')

        

        else:
            # Сигнала нет, монета {coin}
            time.sleep(5)    

# Запустить бот

In [39]:
while True:
    bot()

Поиск монеты....


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:41<00:11, 11.49s/it]

Поиск точки входа по монете: BANDUSDT
Поиск точки входа по монете: BANDUSDT
Открыта позиция в лонг по монете: BANDUSDT, цена: 1.853, время: 2023-04-13 08:30:03
Позиция закрыта по промежуточному ТЭЙКУ
Закрыта позиция в лонг по промежуточному ТЭЙКУ, монета: BANDUSDT, цена: 1.868, прибыль: 0.4, время: 2023-04-13 10:21:31
Позиция закрыта по TAKE_PROFIT
Закрыта позиция в лонг по TAKE_PROFIT, монета: BANDUSDT, цена: 1.898, прибыль: 1.2142471667566102, время: 2023-04-13 16:14:29


100%|██████████████████████████████████████████████████████████████████████████████| 20/20 [7:48:46<00:00, 1406.34s/it]


Поиск монеты....


...

Поиск точки входа по монете: BATUSDT
Поиск точки входа по монете: BATUSDT
Поиск точки входа по монете: BATUSDT
Поиск точки входа по монете: BATUSDT
Поиск точки входа по монете: BATUSDT
Открыта позиция в шорт по монете: BATUSDT, цена: 0.2824, время: 2023-04-14 07:20:05
Позиция закрыта по STOP_LOSS
Закрыта позиция в шорт по STOP_LOSS, монета: BATUSDT, цена: 0.2847, прибыль: -0.8, время: 2023-04-14 07:39:49


  5%|███▉                                                                           | 1/20 [20:54<6:37:18, 1254.66s/it]

Поиск точки входа по монете: APEUSDT
Поиск точки входа по монете: APEUSDT
Поиск точки входа по монете: APEUSDT
Открыта позиция в шорт по монете: APEUSDT, цена: 4.515, время: 2023-04-14 07:40:40


  5%|███▉                                                                           | 1/20 [30:33<9:40:36, 1833.52s/it]


ReadTimeout: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=10)